# Building a Neural Language Model 
Following [Bengio et al 2003](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

## Architecture

The architecture of the MLP we'll be building in this notebook

![hello](neural_probabalistic_language_model_architecture.png "MLP Architecture")

In [2]:
import torch
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [12]:
# build the dataset

block_size = 3
X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


From just 5 words we can create a dataset of 32 examples using `.` as padding characters.

In [13]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [15]:
# We now initialise the Tensor for each character in our vocabulary (27) and it's corresponding embedding.
# For now in 2D as we only have 27 characters
# In Bengio et al, they had 17,000 words and embedded in as small as a 30 dimensional space
C = torch.randn((27, 2))
C

tensor([[ 1.1212,  0.1038],
        [-1.6709,  0.5939],
        [-0.2411, -0.7757],
        [ 0.1091, -0.4565],
        [ 1.2677, -2.0133],
        [ 1.2921,  0.2726],
        [-0.0924, -0.9406],
        [-0.6139,  0.4221],
        [-0.8019, -1.5593],
        [ 0.8555,  0.2140],
        [ 0.7285,  0.1717],
        [-1.6290,  0.5480],
        [ 0.1451,  2.2464],
        [-2.8325,  1.4847],
        [-0.0165,  0.5479],
        [-0.6087,  1.4239],
        [-0.1148, -1.6994],
        [ 0.5736, -0.7081],
        [ 1.4550,  1.6645],
        [ 0.0562, -2.2055],
        [-0.7893,  0.7932],
        [ 0.9540, -0.0100],
        [ 0.0269, -0.8091],
        [-0.1257, -0.5199],
        [-0.6768, -0.0090],
        [ 0.4176, -0.7839],
        [-0.0574,  0.1648]])

Let's try and embed a single integer first

In [16]:
# for 5:
C[5]

tensor([1.2921, 0.2726])

In [18]:
# Using one-hot ecoding
# F.one_hot(5, num_classes=27) # Doesn't work, needs to be a tensor
F.one_hot(torch.tensor(5), num_classes=27)

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

In [21]:
# Doesn't work since the onehot encoded 
# F.one_hot(torch.tensor(5), num_classes=27) @ C
# RuntimeError: expected m1 and m2 to have the same dtype, but got: long long != float

In [22]:
# PyTorch needs to have a float Tensor to know how to multiply with another float Tensor
# self.long() is equivalent to self.to(torch.int64).
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([1.2921, 0.2726])

We get the same embedding as before when we grabbed the 5th index of the Tensor C. 

Therefore in the lookup table highlighted below, we can retrieve the embedding for a character/word/token either by indexing into the tensor of embeddings or by one-hot encoding the vocabulary and then doing an efficient matrix multiplication.

![asd](neural_probabalistic_language_model_architecture_lookup_table.png "Character lookup table")

Here we will index as it is much faster and use a character table.

In [24]:
# We can also index with a list or tensor
C[[5, 6, 7]]

tensor([[ 1.2921,  0.2726],
        [-0.0924, -0.9406],
        [-0.6139,  0.4221]])

In [26]:
C[torch.tensor([5, 6, 7, 7, 7, 7, 7])] # Adding the same index will replicate it multiple times

tensor([[ 1.2921,  0.2726],
        [-0.0924, -0.9406],
        [-0.6139,  0.4221],
        [-0.6139,  0.4221],
        [-0.6139,  0.4221],
        [-0.6139,  0.4221],
        [-0.6139,  0.4221]])

We can also index with a multi-dimensional tensor like out input data `X`

In [28]:
C[X].shape, C[X]

(torch.Size([32, 3, 2]),
 tensor([[[ 1.1212,  0.1038],
          [ 1.1212,  0.1038],
          [ 1.1212,  0.1038]],
 
         [[ 1.1212,  0.1038],
          [ 1.1212,  0.1038],
          [ 1.2921,  0.2726]],
 
         [[ 1.1212,  0.1038],
          [ 1.2921,  0.2726],
          [-2.8325,  1.4847]],
 
         [[ 1.2921,  0.2726],
          [-2.8325,  1.4847],
          [-2.8325,  1.4847]],
 
         [[-2.8325,  1.4847],
          [-2.8325,  1.4847],
          [-1.6709,  0.5939]],
 
         [[ 1.1212,  0.1038],
          [ 1.1212,  0.1038],
          [ 1.1212,  0.1038]],
 
         [[ 1.1212,  0.1038],
          [ 1.1212,  0.1038],
          [-0.6087,  1.4239]],
 
         [[ 1.1212,  0.1038],
          [-0.6087,  1.4239],
          [ 0.1451,  2.2464]],
 
         [[-0.6087,  1.4239],
          [ 0.1451,  2.2464],
          [ 0.8555,  0.2140]],
 
         [[ 0.1451,  2.2464],
          [ 0.8555,  0.2140],
          [ 0.0269, -0.8091]],
 
         [[ 0.8555,  0.2140],
          [ 0.0

In [30]:
X[13, 2]

tensor(1)

In [31]:
C[X][13, 2]

tensor([-1.6709,  0.5939])

In [34]:
C[1]

tensor([-1.6709,  0.5939])

Teherefore we can easily embed all of the input data X by using C[X]

In [35]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [37]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [40]:
# The below doesn't work since emb has shape 32x3x2 and W1 has shape 6x100
# emb @ W1 + b1 
# How do we transform this so that it does work?

So we now need to concatenate the embeddings for each of the context words together to pass through the network.

![ads](neural_probabalistic_language_model_architecture_input_embeddings.png "Input embeddings")

To do this, we can use efficient PyTorch concatenating

In [42]:
emb.shape

torch.Size([32, 3, 2])

1st dimension 32 = number of input examples

2nd dimension 3 = context size (words in the context of the network)

3rd dimension 2 = dimensionality of the embeddings

We need to concatenate the 2nd dimension here together so that the final dimension is 32x6

In [45]:
torch.concat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]]).shape

torch.Size([96, 2])